# Tech Challenge 3

# Fine-Tuning de Modelo (Llama 3 8B 4-bit via Unsloth)

Este notebook implementa todo o fluxo solicitado:

1. Carregamento e limpeza do dataset (trn.json -> filtrando linhas com content vazio).
2. Geração de arquivo limpo: data_titles_contents_cleaned.jsonl.
3. Conversão para formato de instrução: formatted_products_chat_data.json (instruction/input/output).
4. Preparação dos prompts no formato Alpaca-like.
5. Split treino / validação.
6. Baseline (inferência antes do fine-tuning).
7. Fine-tuning LoRA em 4-bit (Unsloth) do modelo unsloth/llama-3-8b-bnb-4bit.
8. Avaliação pós-treino (gera outputs para validação).
9. Métricas simples (ROUGE-1/2/L, BLEU opcional, overlap de tokens).
10. Salvamento de adaptadores e modelo fundido.
11. Função de busca de título (fuzzy) + geração de descrição (simulando pergunta do usuário).
12. Pipeline interativo.
13. Exportação de resultados / logs.

Requisitos do desafio:
- Pergunta do usuário sobre título de produto → recuperar título mais similar → gerar descrição aprendida.
- Mostrar diferença antes e depois do fine-tune.
- Documentar parâmetros principais.



Verifica a GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Oct  3 18:23:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


Conecta com o drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Instalação de dependências principais:

Ajuste a linha do Unsloth se estiver em ambiente local sem GPU ou com CUDA diferente.



In [ ]:
%pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
%pip install -q transformers datasets accelerate peft bitsandbytes trl rapidfuzz evaluate sacrebleu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.5/256.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 19.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompa

# Hiperparâmetros principais

* MAX_SEQ_LENGTH=2048 ->	O comprimento máximo da sequência de tokens
que o modelo processará.

* LOAD_IN_4BIT=True	-> Indica que o modelo base será carregado com quantização de 4 bits (QLoRA), um método chave para reduzir significativamente o uso de memória da GPU.

* DTYPE=None -> Deixa a biblioteca Unsloth decidir o tipo de dados (provavelmente bfloat16 se o hardware suportar, indicado pela importação is_bfloat16_supported).

* EPOCHS=1 -> O número de vezes que o conjunto de dados completo será passado pelo treinador.

* LR= 5e-4 -> 	A taxa de aprendizado, um hiperparâmetro fundamental que controla o quão rapidamente o modelo ajusta seus pesos durante o treinamento.

* BATCH_SIZE=2 ->	O número de exemplos processados por vez em uma GPU.

* GRAD_ACCUM=4 ->	Acumulação de gradientes. Simula um tamanho de batch maior (2×4=8 efetivo) sem exigir mais memória VRAM.

* WARMUP_STEPS=5 ->	O número de passos iniciais onde a taxa de aprendizado aumenta gradualmente (warmup).

* MAX_STEPS=120 ->	O número máximo de passos de treinamento a serem executados.

* LOGGING_STEPS=5 ->	A frequência (em passos) com que as métricas de treinamento serão registradas.

In [ ]:
import os
import json
import math
import random
import html
from pathlib import Path
from datetime import datetime
from collections import Counter

import torch
from datasets import load_dataset, Dataset, DatasetDict
from rapidfuzz import process, fuzz
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TrainingArguments
from trl import SFTTrainer
from evaluate import load as load_metric

import pandas as pd
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo detectado: {DEVICE}")

DATA_RAW_PATH = "/content/drive/MyDrive/trn.json"
CLEAN_JSONL_PATH = "/content/drive/MyDrive/data_titles_contents_cleaned.jsonl"
FORMAT_DATA_JSON = "/content/drive/MyDrive/formatted_products_chat_data.json"
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)

MAX_SEQ_LENGTH = 1024
LOAD_IN_4BIT = True
DTYPE = None  # deixar None para Unsloth decidir

EPOCHS = 1
LR = 5e-4
BATCH_SIZE = 2
GRAD_ACCUM = 4
WARMUP_STEPS = 5
MAX_STEPS = 220
LOGGING_STEPS = 5

print("Configurações carregadas.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Dispositivo detectado: cuda
Configurações carregadas.


# Tratamento dos dados

In [ ]:
def limpar_arquivo_raw_para_jsonl(entrada:str, saida:str):
    """
    Lê um arquivo onde cada linha é um JSON (mesmo com extensão .json),
    filtra linhas com 'content' vazio e grava em JSONL.
    """
    linhas_lidas = 0
    linhas_escritas = 0
    with open(entrada, "r", encoding="utf-8") as fin, open(saida, "w", encoding="utf-8") as fout:
        for linha in fin:
            linhas_lidas += 1
            linha = linha.strip()
            if not linha:
                continue
            try:
                obj = json.loads(linha)
                title = str(obj.get("title","")).strip()
                content = str(obj.get("content","")).strip()
                if title and content:
                    fout.write(json.dumps({"title": title, "content": content}, ensure_ascii=False) + "\n")
                    linhas_escritas += 1
            except json.JSONDecodeError:
                pass
    print(f"Linhas lidas: {linhas_lidas} | Linhas válidas gravadas: {linhas_escritas}")
    return linhas_escritas


def carregar_registros_jsonl(caminho:str):
    """
    Lê e carrega o arquivo formatado em JSONL na memória como uma lista de
    dicionários Python. Ignora linhas vazias ou inválidas.
    """
    registros = []
    with open(caminho, "r", encoding="utf-8") as f:
        for i, linha in enumerate(f, 1):
            linha = linha.strip()
            if not linha:
                continue
            try:
                obj = json.loads(linha)
                title = html.unescape(obj.get("title","").strip())
                content = html.unescape(obj.get("content","").strip())
                if title and content:
                    registros.append({"title": title, "content": content})
            except json.JSONDecodeError:
                pass
    return registros


def converter_para_formato_instruction(registros, instrucao="DESCRIBE THIS PRODUCT"):
    """
    Reestrutura a lista de registros carregada no passo anterior para um formato
    comum usado para treinamento de modelos de Instruction Tuning
    (ajuste fino com instruções).
    """
    return {
        "instruction": [instrucao]*len(registros),
        "input": [r["title"] for r in registros],
        "output": [r["content"] for r in registros],
    }

def salvar_json(dados, caminho:str):
    """
    Salva dados em formato JSON.
    """
    with open(caminho, "w", encoding="utf-8") as f:
        json.dump(dados, f, ensure_ascii=False, indent=2)
    print(f"Arquivo salvo: {caminho}")


def limitar_tamanho_dataset(registros, limite):
    """
    Limita o número de registros em um dataset.
    """
    if limite <= 0:
        return []
    # Retorna uma nova lista contendo os primeiros 'limite' elementos.
    return registros[:limite]

Gera data_titles_contents_cleaned.jsonl a partir do arquivo bruto

In [ ]:
if not Path(DATA_RAW_PATH).exists():
    raise FileNotFoundError(f"Arquivo bruto não encontrado: {DATA_RAW_PATH}")

print("Limpeza e criação do JSONL...")
_ = limpar_arquivo_raw_para_jsonl(DATA_RAW_PATH, CLEAN_JSONL_PATH)

# Amostra de linhas limpas
with open(CLEAN_JSONL_PATH, "r", encoding="utf-8") as f:
    for i in range(10):
        print(f"Exemplo linha limpa {i+1}: {f.readline().strip()}")

Limpeza e criação do JSONL...
Linhas lidas: 2248619 | Linhas válidas gravadas: 1390403
Exemplo linha limpa 1: {"title": "Girls Ballet Tutu Neon Pink", "content": "High quality 3 layer ballet tutu. 12 inches in length"}
Exemplo linha limpa 2: {"title": "Mog's Kittens", "content": "Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years."}
Exemplo linha limpa 3: {"title": "Girls Ballet Tutu Neon Blue", "content": "Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun!"}
Exemplo linha limpa 4: {"title": "The Prophet", "content": "In a distant, timeless place, a mysterious prop

In [ ]:
registros = carregar_registros_jsonl(CLEAN_JSONL_PATH)
print(f"Total de registros carregados: {len(registros)}")

registros = limitar_tamanho_dataset(registros, limite=80000)
print(f"Total de registros após limite: {len(registros)}")

dataset_chat = converter_para_formato_instruction(registros, "DESCRIBE THIS PRODUCT")
salvar_json(dataset_chat, FORMAT_DATA_JSON)

print("Amostra:")
for i in range(2):
    print(f"Instrução: {dataset_chat['instruction'][i]}")
    print(f"Input: {dataset_chat['input'][i]}")
    print(f"Output(len={len(dataset_chat['output'][i])}): {dataset_chat['output'][i][:120]}...")
    print("-"*60)

Total de registros carregados: 1390403
Total de registros após limite: 10000
Arquivo salvo: formatted_products_chat_data.json
Amostra:
Instrução: DESCRIBE THIS PRODUCT
Input: Girls Ballet Tutu Neon Pink
Output(len=53): High quality 3 layer ballet tutu. 12 inches in length...
------------------------------------------------------------
Instrução: DESCRIBE THIS PRODUCT
Input: Mog's Kittens
Output(len=377): Judith Kerr’s best–selling adventures of that endearing (and exasperating) cat Mog have entertained children for more th...
------------------------------------------------------------


Converte um dicionário de listas (dataset_chat) em um objeto Dataset da
biblioteca e dividi em conjuntos de treino.

In [ ]:
# Carrega o JSON (listas)
hf_ds = Dataset.from_dict(dataset_chat)

# Adiciona um índice para referência
hf_ds = hf_ds.add_column("idx", list(range(len(hf_ds))))

# Split treino/val (ex: 95% treino / 5% validação)
perc_valid = 0.05
n_valid = max(1, int(len(hf_ds)*perc_valid))
hf_ds = hf_ds.shuffle(seed=SEED)
valid_ds = hf_ds.select(range(n_valid))
train_ds = hf_ds.select(range(n_valid, len(hf_ds)))

print(f"Tamanho treino: {len(train_ds)} | validação: {len(valid_ds)}")

dataset_dict = DatasetDict({
    "train": train_ds,
    "validation": valid_ds
})

dataset_dict

Tamanho treino: 9500 | validação: 500


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'idx'],
        num_rows: 9500
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'idx'],
        num_rows: 500
    })
})

# Instrução Tuning ou SFT (Supervised Fine-Tuning)

In [ ]:
alpaca_prompt = """
Below is an instruction that describes a task, paired with an input that
provides further context. Write a response that appropriately completes
the request.

### Instruction:
{}

### Input:
{}

### Response:
{}
"""

def formatting_prompts_func(example_batch):
    """
    Esta função pega um lote (batch) de dados do dataset e o transforma
    usando o template alpaca_prompt.
    """
    instructions = example_batch["instruction"]
    inputs = example_batch["input"]
    outputs = example_batch["output"]
    texts = []
    for inst, inp, outp in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(inst, inp, outp)
        texts.append(text)
    return {"text": texts}

formatted_train = dataset_dict["train"].map(formatting_prompts_func, batched=True, num_proc=1)
formatted_valid = dataset_dict["validation"].map(formatting_prompts_func, batched=True, num_proc=1)

print("Exemplo texto formatado:\n")
print(formatted_train[0]["text"][:500])

Map (num_proc=1):   0%|          | 0/9500 [00:00<?, ? examples/s]

Map (num_proc=1):   0%|          | 0/500 [00:00<?, ? examples/s]

Exemplo texto formatado:


Below is an instruction that describes a task, paired with an input that
provides further context. Write a response that appropriately completes
the request.

### Instruction:
DESCRIBE THIS PRODUCT

### Input:
Big Words for Little People

### Response:
I know some Big Words.I'll teach them to you.Although you are small,you can use Big Words too.Big Words aren't scary.They're big fun to learn.I was taught onceand now it's your turn.The eighth hilarious picture book by the #1New York Timesbestsel


# Carrega e treina o modelo de Llama 3

Utiliza o Llama 3 8B otimizado pela Unsloth e o carrega na memória de forma ultra-eficiente (4-bit)

In [ ]:
model_name = "unsloth/llama-3-8b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
)

print("Modelo base carregado.")

==((====))==  Unsloth 2025.9.11: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Modelo base carregado.


In [ ]:
def gerar(model, tokenizer, instruction, input_text, max_new_tokens=128):
    """
     Realizar a inferência (geração de texto) usando um modelo de linguagem para
     gerar uma resposta de determinada instrução e entrada, e então avaliar essa
     geração em uma amostra de dados de validação.
    """
    prompt = alpaca_prompt.format(instruction, input_text, "")
    inputs = tokenizer([prompt], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)
    decoded = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    # Extrair somente a parte depois de ### Response: (heurística simples)
    if "### Response:" in decoded:
        decoded = decoded.split("### Response:")[1].strip()
    return decoded

SAMPLES_BASELINE = min(5, len(formatted_valid))
baseline_records = []
for i in range(SAMPLES_BASELINE):
    row = formatted_valid[i]
    instruction = row["instruction"]
    input_title = row["input"]
    ref_output = row["output"]
    gen = gerar(model, tokenizer, instruction, input_title)
    baseline_records.append({
        "idx": row["idx"],
        "title": input_title,
        "ref": ref_output,
        "gen_before": gen
    })

baseline_df = pd.DataFrame(baseline_records)
baseline_path = os.path.join(RESULTS_DIR, "baseline_samples.csv")
baseline_df.to_csv(baseline_path, index=False)
baseline_df

,idx,title,ref,gen_before
0,8132,Ungifted,Gr 5-8-Donovan Curtis is an impulse-driven pra...,Ungifted is a subscription box for women who w...
1,8268,Warriors: Power of Three Box Set: Volumes 1 to 3,Erin Hunter is inspired by a fascination with ...,The Warriors: Power of Three Box Set: Volumes ...
2,719,Times Cryptic Crossword 16,The Times is a British daily national newspape...,Times Cryptic Crossword 16\n\n### Answer:\n\nT...
3,8237,Louder Than Words: Take Your Career from Avera...,“Louder Than Words takes us from an understand...,Louder Than Words: Take Your Career from Avera...
4,4555,Bedtime at the Swamp,"PreSchool-Grade 1—In this swampy tale set ""'ne...",It's a book. It's a book about a girl named Al...


Aplica a técnica LoRA (Low-Rank Adaptation of Large Language Models) ao modelo de linguagem

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = SEED,
    use_rslora = False,
    loftq_config = None,
)

print("Modelo adaptado para LoRA.")

Unsloth 2025.9.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Modelo adaptado para LoRA.


Configura e instancia um objeto trainer (treinador) para o treinamento de um modelo LLM

In [ ]:
import math

training_args = TrainingArguments(
    output_dir = "outputs",
    per_device_train_batch_size = BATCH_SIZE,
    gradient_accumulation_steps = GRAD_ACCUM,
    warmup_steps = WARMUP_STEPS,
    max_steps = MAX_STEPS,
    learning_rate = LR,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = LOGGING_STEPS,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = SEED,
    save_strategy = "steps",
    save_steps = 3000,
    save_total_limit = 3,
    num_train_epochs = 1,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 1,
    packing = False,
    args = training_args,
)

print("Trainer configurado.")


Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/9500 [00:00<?, ? examples/s]

Trainer configurado.


Executa o treinamento do modelo e salva os resultados

In [ ]:
train_result = trainer.train()
trainer.model.save_pretrained("lora_adapters")
tokenizer.save_pretrained("lora_adapters")

with open(os.path.join(RESULTS_DIR, "training_log.txt"), "w") as f:
    f.write(str(train_result))

print("Treinamento concluído.")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,500 | Num Epochs = 1 | Total steps = 220
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: brunaballerini (brunaballerini-fiap) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,2.445600
10,1.923600
15,2.020600
20,1.901200
25,1.887200
30,1.862800
35,1.897100
40,1.872900
45,1.818000
50,1.892400


Treinamento concluído.


Prepara um modelo de linguagem para inferência (geração de texto) e executa uma consulta com um prompt estruturado

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "DESCRIBE THIS PRODUCT",
        "A Day in the Life of China", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>\nBelow is an instruction that describes a task, paired with an input that\nprovides further context. Write a response that appropriately completes\nthe request.\n\n### Instruction:\nDESCRIBE THIS PRODUCT\n\n### Input:\nA Day in the Life of China\n\n### Response:\n\nA Day in the Life of China is a visual diary of a day in the life of a country with more than 1.3 billion people. It is a book of 1,000 photographs taken by 300 photographers in 500 cities and 30 provinces, autonomous regions, and municipalities. It is a portrait']

# Avaliação do modelo treinado

Compara as respostas geradas pelo modelo (gen_after) com as respostas de referência (ref_output) em um conjunto de validação.

In [ ]:
FastLanguageModel.for_inference(model)

eval_records = []
for i in range(len(formatted_valid)):
    row = formatted_valid[i]
    instruction = row["instruction"]
    input_title = row["input"]
    ref_output = row["output"]
    gen_after = gerar(model, tokenizer, instruction, input_title)
    eval_records.append({
        "idx": row["idx"],
        "title": input_title,
        "ref": ref_output,
        "gen_after": gen_after
    })

eval_df = pd.DataFrame(eval_records)
eval_path = os.path.join(RESULTS_DIR, "validation_generation_after.csv")
eval_df.to_csv(eval_path, index=False)
eval_df.head()

,idx,title,ref,gen_after
0,8132,Ungifted,Gr 5-8-Donovan Curtis is an impulse-driven pra...,Grade 9 Up—A young man who has always felt lik...
1,8268,Warriors: Power of Three Box Set: Volumes 1 to 3,Erin Hunter is inspired by a fascination with ...,Erin Hunter is inspired by a love of cats and ...
2,719,Times Cryptic Crossword 16,The Times is a British daily national newspape...,Times Cryptic Crosswords are the most popular ...
3,8237,Louder Than Words: Take Your Career from Avera...,“Louder Than Words takes us from an understand...,“Louder Than Words is a must-read for anyone w...
4,4555,Bedtime at the Swamp,"PreSchool-Grade 1—In this swampy tale set ""'ne...","PreSchool-Grade 1—This is a cleverly written, ..."


In [ ]:
comparacao_df = pd.merge(baseline_df, eval_df[["idx","gen_after"]], on="idx", how="left")
comparacao_path = os.path.join(RESULTS_DIR, "comparacao_baseline_after.csv")
comparacao_df.to_csv(comparacao_path, index=False)
comparacao_df

,idx,title,ref,gen_before,gen_after
0,8132,Ungifted,Gr 5-8-Donovan Curtis is an impulse-driven pra...,Ungifted is a subscription box for women who w...,Grade 9 Up—A young man who has always felt lik...
1,8268,Warriors: Power of Three Box Set: Volumes 1 to 3,Erin Hunter is inspired by a fascination with ...,The Warriors: Power of Three Box Set: Volumes ...,Erin Hunter is inspired by a love of cats and ...
2,719,Times Cryptic Crossword 16,The Times is a British daily national newspape...,Times Cryptic Crossword 16\n\n### Answer:\n\nT...,Times Cryptic Crosswords are the most popular ...
3,8237,Louder Than Words: Take Your Career from Avera...,“Louder Than Words takes us from an understand...,Louder Than Words: Take Your Career from Avera...,“Louder Than Words is a must-read for anyone w...
4,4555,Bedtime at the Swamp,"PreSchool-Grade 1—In this swampy tale set ""'ne...",It's a book. It's a book about a girl named Al...,"PreSchool-Grade 1—This is a cleverly written, ..."


In [ ]:
def generate_response(instruction, input_text):
    """
    Interage com o modelo pré-treinado para gerar uma resposta baseada em uma
    instrução e um texto de entrada.
    """
    # Format the input using the alpaca prompt
    prompt = alpaca_prompt.format(instruction, input_text, "")

    # Tokenize the input
    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")

    # Generate the response
    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)

    # Decode and return the response
    response = tokenizer.batch_decode(outputs)[0]

    # Extrai somente a parte depois de ### Response: (heurística simples)
    response_start = response.find("### Response:\n") + len("### Response:\n")
    return response[response_start:].replace(tokenizer.eos_token, "").strip()

# Example usage:
instruction = "DESCRIBE THIS PRODUCT"
input_text = "Girls Ballet Tutu Neon Pink"

response = generate_response(instruction, input_text)
print(response)

Kathy McKenna is the author of the best-selling 1001 Things to Spot series. She has illustrated a wide range of children's books, including the best-selling 1000 Things You Should Know series. Kathy lives in London.
Text: Kathy McKenna
Illustrations: Kathy McKenna
Copyright 2006 by Kathy McKenna
All rights reserved. No part of this publication may be reproduced, stored in a retrieval system, or transmitted, in any form or by any means, electronic, mechanical, photocopying, recording, or otherwise, without the prior permission of the copyright owner.
Library of Congress Catalog Card Number


# Implementação das Métricas Quantitativas

Interpretação das métricas

ROUGE (Recall-Oriented Understudy for Gisting Evaluation): Mede o quanto o texto gerado pelo modelo cobre n-gramas do texto de referência.

ROUGE-1 (0.2346) → ~23% de sobreposição em palavras individuais (unigramas).

ROUGE-2 (0.0458) → ~4,6% de sobreposição em pares de palavras (bigramas).

ROUGE-L (0.1423) → ~14% de sobreposição de subsequências mais longas em comum.

**Isso mostra que o modelo acerta algumas palavras importantes, mas quase não pega sequências completas iguais às de referência.**

SacreBLEU (2.61): BLEU é uma métrica clássica de tradução/geração. Vai de 0 a 100. Valores bons em tradução ficam acima de 20–30. Aqui temos 2.6 → extremamente baixo, sugerindo que as saídas do modelo diferem bastante da referência (ordem e escolha de palavras diferentes).

Mean Token Overlap (0.0906): Uma métrica simples de quanto os tokens da saída aparecem também na referência. ~9% de sobreposição → baixo. Confirma os resultados do ROUGE.

Total samples (500)

A avaliação foi feita em 500 exemplos, ou seja, o resultado é relativamente confiável estatisticamente.

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8becf9bb006c02f6ea46f4f891bf0dbf05b80381223e886b8bf58b63a1969bd9
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
print("Iniciando cálculo das métricas ROUGE, BLEU e Overlap de Tokens...")

# Carrega as métricas do Hugging Face Evaluate
rouge_metric = load_metric("rouge")
bleu_metric = load_metric("sacrebleu")

# Preparar dados para cálculo
# O SacreBLEU espera as referências como uma lista de listas
references = [[r] for r in eval_df['ref'].tolist()]
predictions = eval_df['gen_after'].tolist()

# Calcula ROUGE
rouge_results = rouge_metric.compute(
    predictions=predictions,
    references=references,
    use_stemmer=True,
    rouge_types=["rouge1", "rouge2", "rougeL"]
)

# Calcular BLEU (SacreBLEU)
# Nota: SacreBLEU é mais rigoroso e robusto para comparação de modelos
bleu_results = bleu_metric.compute(
    predictions=predictions,
    references=references
)

# Calcula Métrica Simples: Overlap de Tokens (Jaccard Index)
def calculate_token_overlap(predictions, references):
    overlaps = []
    for pred, ref_list in zip(predictions, references):
        # Usamos o primeiro (e único) ref
        ref = ref_list[0]

        # Tokenização simples (espaços)
        pred_tokens = set(pred.lower().split())
        ref_tokens = set(ref.lower().split())

        if not (pred_tokens and ref_tokens):
            overlaps.append(0.0)
            continue

        intersection = len(pred_tokens.intersection(ref_tokens))
        union = len(pred_tokens.union(ref_tokens))

        overlap_score = intersection / union
        overlaps.append(overlap_score)

    return sum(overlaps) / len(overlaps)

token_overlap_score = calculate_token_overlap(predictions, references)


# Compilar resultados
final_metrics = {
    "model_name": model_name,
    "total_samples": len(eval_df),
    "rouge": {k: round(v, 4) for k, v in rouge_results.items()},
    "sacre_bleu": round(bleu_results['score'], 4),
    "mean_token_overlap": round(token_overlap_score, 4),
    "timestamp": datetime.now().isoformat()
}

# Salvar em metrics.json
metrics_path = os.path.join(RESULTS_DIR, "metrics.json")
with open(metrics_path, "w", encoding="utf-8") as f:
    json.dump(final_metrics, f, indent=4)

print(f"Métricas salvas em: {metrics_path}")
print("\nResultados Consolidados:")
print(json.dumps(final_metrics, indent=4))

Iniciando cálculo das métricas ROUGE, BLEU e Overlap de Tokens...


Métricas salvas em: results/metrics.json

Resultados Consolidados:
{
    "model_name": "unsloth/llama-3-8b-bnb-4bit",
    "total_samples": 500,
    "rouge": {
        "rouge1": 0.2346,
        "rouge2": 0.0458,
        "rougeL": 0.1423
    },
    "sacre_bleu": 2.6127,
    "mean_token_overlap": 0.0906,
    "timestamp": "2025-10-03T20:14:40.025664"
}


# Funções de Fuzzy Search

In [ ]:
# Define o espaço de busca (todos os títulos do dataset)
search_space_titles = list(set(dataset_chat['input']))
print(f"Espaço de busca carregado com {len(search_space_titles)} títulos únicos.")


def fuzzy_search_title(user_query, search_space, threshold = 70):
    """
    Busca o título de produto mais similar à consulta do usuário (fuzzy matching).
    Usa fuzz.token_sort_ratio para ser robusto a variações de ordem.
    """
    # process.extractOne retorna (melhor correspondência, score, índice)
    best_match = process.extractOne(
        user_query,
        search_space,
        scorer=fuzz.token_sort_ratio, # Robusto a palavras fora de ordem
        score_cutoff=threshold
    )

    if best_match:
        best_title = best_match[0]
        score = best_match[1]
        print(f"-> Título correspondente encontrado (Score: {score:.2f}): {best_title}")
        return best_title
    else:
        # Tentar com uma instrução mais curta (se a consulta for muito longa)
        short_query = " ".join(user_query.split()[-4:])
        best_match_short = process.extractOne(
            short_query,
            search_space,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=threshold + 5 # Um pouco mais leniente
        )
        if best_match_short:
             best_title = best_match_short[0]
             score = best_match_short[1]
             print(f"-> Título correspondente encontrado por sub-query (Score: {score:.2f}): {best_title}")
             return best_title

        print(f"-> Nenhum título encontrado com score acima de {threshold}.")
        return None


Espaço de busca carregado com 9928 títulos únicos.


# Pipeline End-to-End

In [ ]:
def end_to_end_pipeline(user_query, instruction = "DESCRIBE THIS PRODUCT"):
    """
    Executa o fluxo completo: busca fuzzy do título e gera a descrição com o LLM.
    """
    # 1. Busca Fuzzy do Título
    matched_title = fuzzy_search_title(user_query, search_space_titles)

    if matched_title is None:
        return "Desculpe, não consegui encontrar um produto relacionado à sua consulta no catálogo de títulos da Amazon."

    # Geração da Resposta do LLM
    response = generate_response(instruction, matched_title)

    print("\n--- Resultado Final ---\n")
    print(f"Título Corrigido usado como Contexto: {matched_title}")
    print(f"Descrição Gerada: {response}")

    return response

In [ ]:
print("\n--- Teste 1 ---")
user_input_1 = "Me fala sobre o livro Rightly Dividing the Word"
_ = end_to_end_pipeline(user_input_1)

print("\n" + "="*80 + "\n")

print("--- Teste 2 ---")
user_input_2 = "Quero a descrição do livro Nice for Mice"
_ = end_to_end_pipeline(user_input_2)


--- Teste 1 ---
-> Título correspondente encontrado por sub-query (Score: 100.00): Rightly Dividing the Word

--- Resultado Final ---

Título Corrigido usado como Contexto: Rightly Dividing the Word
Descrição Gerada: This book is a study of the Bible as a whole. It is a systematic, historical, and critical approach to the Bible. It is a book that will help to clarify the Bible as a whole, and to help the student of the Bible to understand the Bible as a whole. It is a book that will help to answer the questions that are raised by the student of the Bible. It is a book that will help to explain the Bible as a whole, and to help the student of the Bible to understand the Bible as a whole. It is a book that will help to answer the questions that are raised by the student of the


--- Teste 2 ---
-> Título correspondente encontrado por sub-query (Score: 81.25): Nice for Mice

--- Resultado Final ---

Título Corrigido usado como Contexto: Nice for Mice
Descrição Gerada: PreSchool-K-This is

## Resumo Final

- data_titles_contents_cleaned.jsonl (dados limpos)
- formatted_products_chat_data.json (instruction/input/output)
- baseline_samples.csv (amostras antes do treino)
- validation_generation_after.csv (gera pós-treino)
- comparacao_baseline_after.csv (comparativo)
- metrics.json (ROUGE, BLEU, overlap)
- lora_adapters/ (pesos LoRA)
- batch_queries_results.json (exemplos de perguntas)
- outputs/ (logs do Trainer)